# Abstract Matrices

In [1]:
import finite_algebras as alg
import numpy as np

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

# ex = alg.Examples(alg_dir)

In [2]:
ps3 = alg.generate_powerset_ring(3)
ps3.about()


** Ring **
Name: PSRing3
Instance ID: 4374640080
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
Mult. Identity: {0, 1, 2}
Mult. Commutative? Yes
Zero Divisors: ['{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0,

In [3]:
ps3.zero

'{}'

In [4]:
class AbstractMatrix:

    def __init__(self, array, ring):
        if isinstance(array, np.ndarray):
            arr = array
        else:
            arr = np.array(array)
        self.__ring = ring
        self.__array = np.array(array, dtype='<U32')
    
    @classmethod
    def zeros(cls, shape, ring):
        arr = np.full(shape, ring.zero, dtype='<U32')
        return cls(arr, ring)
    
    @classmethod
    def random(cls, shape, ring):
        rand_indices = np.random.randint(ring.order, size=shape)
        rand_array = np.full(shape, ring.zero, dtype='<U32')
        for i in range(shape[0]):
            for j in range(shape[1]):
                rand_array[i, j] = ring.elements[rand_indices[i, j]]
        return cls(rand_array, ring)
    
    def get_array(self):
        return self.__array
    
    def get_shape(self):
        return self.__array.shape
    
    def get_nrows(self):
        return self.__array.shape[0]
    
    def get_ncols(self):
        return self.__array.shape[1]
    
    def get_algebra(self):
        return self.__ring
    
    def copy(self):
        return AbstractMatrix(np.copy(self.__array), self.__ring)
    
    def transpose(self):
        return AbstractMatrix(np.transpose(self.__array), self.__ring)
    
    def __mul__(self, other):  # Matrix multiplication using Ring operations
        # X * Y
        xarr =  self.__array
        yarr = other.__array
        xrows = self.get_nrows()
        xcols = self.get_ncols()
        yrows = other.get_nrows()
        ycols = other.get_ncols()
        product = None
        if xcols == yrows:
            if self.__ring == other.__ring:
                ring = self.__ring
                product = np.full((xrows, ycols), ring.zero, dtype='U32')
                for i in range(xrows):
                    for j in range(ycols):
                        for k in range(xcols):
                            product[i, j] = ring.add(product[i, j], ring.mult(xarr[i, k], yarr[k, j]))
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are incompatible: {xcols} colums vs {yrows} rows")
        return AbstractMatrix(product, ring)
    
    def __add__(self, other):  # Matrix addition using Ring operations
        # X + Y
        xarr =  self.__array
        yarr = other.__array
        xshape = xarr.shape
        yshape = yarr.shape
        sum = None
        if xshape == yshape:
            if self.__ring == other.__ring:
                ring = self.__ring
                sum = np.full(xshape, ring.zero, dtype='U32')
                for i in range(xshape[0]):
                    for j in range(xshape[1]):
                        sum[i, j] = ring.add(xarr[i, j], yarr[i, j])
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are not equal: {xshape} != {yshape}")
        return AbstractMatrix(sum, ring)
    
    def __sub__(self, other):  # Matrix subtraction using Ring operations
        # X - Y
        xarr =  self.__array
        yarr = other.__array
        xshape = xarr.shape
        yshape = yarr.shape
        sum = None
        if xshape == yshape:
            if self.__ring == other.__ring:
                ring = self.__ring
                sum = np.full(xshape, ring.zero, dtype='U32')
                for i in range(xshape[0]):
                    for j in range(xshape[1]):
                        sum[i, j] = ring.sub(xarr[i, j], yarr[i, j])
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are not equal: {xshape} != {yshape}")
        return AbstractMatrix(sum, ring)

In [5]:
mat1 = AbstractMatrix.zeros((2, 3), ps3)

arr = mat1.get_array()
arr

array([['{}', '{}', '{}'],
       ['{}', '{}', '{}']], dtype='<U32')

In [6]:
fubar = [['{0}', '{1}', '{2}'],
         ['{2}', '{1}', '{0}']]

mat2 = AbstractMatrix(fubar, ps3)

mat2.get_array()

array([['{0}', '{1}', '{2}'],
       ['{2}', '{1}', '{0}']], dtype='<U32')

In [7]:
mat3 = mat2.transpose()
mat3.get_array()

array([['{0}', '{2}'],
       ['{1}', '{1}'],
       ['{2}', '{0}']], dtype='<U32')

In [8]:
mat2x3 = mat2 * mat3
mat2x3.get_array()

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

In [9]:
mat3x2 = mat3 * mat2
mat3x2.get_array()

array([['{0, 2}', '{}', '{}'],
       ['{}', '{}', '{}'],
       ['{}', '{}', '{0, 2}']], dtype='<U32')

In [10]:
try:
    mat2x2 = mat2 * mat2
    print(mat2x2.get_array())
except Exception as exc:
    print(exc)

The array shapes are incompatible: 3 colums vs 2 rows


In [11]:
mat2p2 = mat2 + mat2

mat2p2.get_array()

array([['{}', '{}', '{}'],
       ['{}', '{}', '{}']], dtype='<U32')

In [12]:
try:
    mat2p3 = mat2 + mat3
    print(mat2p3.get_array())
except Exception as exc:
    print(exc)

The array shapes are not equal: (2, 3) != (3, 2)


In [13]:
mat2m2 = mat2 - mat2
mat2.get_array()

array([['{0}', '{1}', '{2}'],
       ['{2}', '{1}', '{0}']], dtype='<U32')

## Determinant

### Example code

See [Find the Determinant of a Matrix with Pure Python without Numpy or Scipy](https://integratedmlai.com/find-the-determinant-of-a-matrix-with-pure-python-without-numpy-or-scipy/)

or [this github file](https://github.com/ThomIves/BasicLinearAlgebraToolsPurePy/blob/master/LinearAlgebraPurePython.py)

In [14]:
def zeros_matrix(rows, cols):
    """
    Creates a matrix filled with zeros.
        :param rows: the number of rows the matrix should have
        :param cols: the number of columns the matrix should have

        :return: list of lists that form the matrix
    """
    M = []
    while len(M) < rows:
        M.append([])
        while len(M[-1]) < cols:
            M[-1].append(0.0)

    return M

def copy_matrix(M):
    """
    Creates and returns a copy of a matrix.
        :param M: The matrix to be copied

        :return: A copy of the given matrix
    """
    # Section 1: Get matrix dimensions
    rows = len(M)
    cols = len(M[0])

    # Section 2: Create a new matrix of zeros
    MC = zeros_matrix(rows, cols)

    # Section 3: Copy values of M into the copy
    for i in range(rows):
        for j in range(cols):
            MC[i][j] = M[i][j]

    return MC

def determinant_recursive(A, total=0):
    """
    Find determinant of a square matrix using full recursion
        :param A: the matrix to find the determinant for
        :param total=0: safely establish a total at each recursion level

        :returns: the running total for the levels of recursion
    """
    # Section 1: store indices in list for flexible row referencing
    indices = list(range(len(A)))

    # Section 2: when at 2x2 submatrices recursive calls end
    if len(A) == 2 and len(A[0]) == 2:
        val = A[0][0] * A[1][1] - A[1][0] * A[0][1]
        return val

    # Section 3: define submatrix for focus column and call this function
    for fc in indices:  # for each focus column, find the submatrix ...
        As = copy_matrix(A)  # make a copy, and ...
        As = As[1:]  # ... remove the first row
        height = len(As)

        for i in range(height):  # for each remaining row of submatrix ...
            As[i] = As[i][0:fc] + As[i][fc+1:]  # zero focus column elements

        sign = (-1) ** (fc % 2)  # alternate signs for submatrix multiplier
        sub_det = determinant_recursive(As)  # pass submatrix recursively
        total += sign * A[0][fc] * sub_det  # total all returns from recursion

    return total

def determinant_recursive_DEBUG(A, total=0):
    """
    Find determinant of a square matrix using full recursion
        :param A: the matrix to find the determinant for
        :param total=0: safely establish a total at each recursion level

        :returns: the running total for the levels of recursion
    """
    # Section 1: store indices in list for flexible row referencing
    indices = list(range(len(A)))

    # Section 2: when at 2x2 submatrices recursive calls end
    print(f"\nlen(A) = {len(A)}")  # DEBUG PRINT
    if len(A) == 2 and len(A[0]) == 2:
        val = A[0][0] * A[1][1] - A[1][0] * A[0][1]
        return val

    # Section 3: define submatrix for focus column and call this function
    for fc in indices:  # for each focus column, find the submatrix ...
        print(f"fc = {fc}")  # DEBUG PRINT
        As = copy_matrix(A)  # make a copy, and ...
        As = As[1:]  # ... remove the first row
        height = len(As)
        print(f"height = {height}")  # DEBUG PRINT

        for i in range(height):  # for each remaining row of submatrix ...
            print(f"               i = {i}")
            print(f"     As[i][0:fc] = {As[i][0:fc]}")
            print(f"    As[i][fc+1:] = {As[i][fc+1:]}")
            As[i] = As[i][0:fc] + As[i][fc+1:]  # zero focus column elements
            print(f"           As[i] = {As[i]}")

        sign = (-1) ** (fc % 2)  # alternate signs for submatrix multiplier
        sub_det = determinant_recursive_DEBUG(As)  # pass submatrix recursively
        total += sign * A[0][fc] * sub_det  # total all returns from recursion

    return total

In [15]:
test1 = [[1, 2, 1],
         [0, 3, 4],
         [3, 1, 4]]

determinant_recursive_DEBUG(test1)


len(A) = 3
fc = 0
height = 2
               i = 0
     As[i][0:fc] = []
    As[i][fc+1:] = [3, 4]
           As[i] = [3, 4]
               i = 1
     As[i][0:fc] = []
    As[i][fc+1:] = [1, 4]
           As[i] = [1, 4]

len(A) = 2
fc = 1
height = 2
               i = 0
     As[i][0:fc] = [0]
    As[i][fc+1:] = [4]
           As[i] = [0, 4]
               i = 1
     As[i][0:fc] = [3]
    As[i][fc+1:] = [4]
           As[i] = [3, 4]

len(A) = 2
fc = 2
height = 2
               i = 0
     As[i][0:fc] = [0, 3]
    As[i][fc+1:] = []
           As[i] = [0, 3]
               i = 1
     As[i][0:fc] = [3, 1]
    As[i][fc+1:] = []
           As[i] = [3, 1]

len(A) = 2


23

## Abstract Determinant

The code below applies to a square abstract numpy array, not an abstract matrix.

In [59]:
def determinant(array, ring, det=0):
    
    ncols = array.shape[1]
    col_indices = list(range(ncols))

    # Recursion ends when only 2x2 matrices remain
    if ncols == 2:
        value = ring.sub(ring.mult(array[0][0], array[1][1]), ring.mult(array[1][0], array[0][1]))
        return value

    arr = np.delete(array, 0, 0)  # Copy array & delete the 1st row
    print(f"arr = {arr}")
    
    for idx in col_indices:
        sub_arr = np.delete(arr, idx, 1)  # Copy arr & delete the column idx
        print(f"sub_arr = {sub_arr}")
        sub_det = determinant(sub_arr, ring)  # Recurse
        print(f"sub_det = {sub_det}")
        if (-1) ** (idx % 2) == 1:  # Alternate addition & substraction
            det = ring.add(det, ring.mult(array[0][idx], sub_det))
            print(f"  add: det = {det}")
        elif sign == -1:
            det = ring.sub(det, ring.mult(array[0][idx], sub_det))
            print(f"  sub: det = {det}")
        else:
            raise ValueError(f"Unexpected value of sign: {sign}")

    return det

### 2x2 Test

In [60]:
test2 = mat2x3.get_array()
test2

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

In [61]:
determinant(test2, mat2x3.get_algebra())

'{0, 2}'

In [62]:
ps3.mult('{0, 1, 2}', '{0, 1, 2}')

'{0, 1, 2}'

In [63]:
ps3.mult('{1}', '{1}')

'{1}'

In [64]:
ps3.sub('{0, 1, 2}', '{1}')

'{0, 2}'

### 3x3 Test

In [65]:
rnd1 = AbstractMatrix.random((3,3),ps3)
rnd1.get_array()

array([['{1}', '{0, 2}', '{0}'],
       ['{0, 1}', '{1}', '{2}'],
       ['{1, 2}', '{1, 2}', '{0, 1}']], dtype='<U32')

In [66]:
determinant(rnd1.get_array(), ps3)

arr = [['{0, 1}' '{1}' '{2}']
 ['{1, 2}' '{1, 2}' '{0, 1}']]
sub_arr = [['{1}' '{2}']
 ['{1, 2}' '{0, 1}']]
sub_det = {1, 2}


ValueError: 0 is not in list